In [1]:
%pip install openpyxl



     |████████████████████████████████| 243 kB 911 kB/s 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import RobustScaler, MinMaxScaler,StandardScaler



In [3]:
data = pd.read_excel('../input/update002/1-s2.0-S0022231321001368-mmc1.xlsx', index_col=0)
data = data.drop(['Link'], axis=1)
df = data.copy()


In [4]:
df.head()


,TeO2,ZnO,Na2O,Er2O3,TiO2,BaO,Na2CO3,GeO2,B2O3,MgO,...,Nb2O5,PbO,PbCl2,PbBr2,PbF2,CaO,P2O5,W2,W4,W6
S1,80.721157,11.931813,4.543149,2.803881,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.14,2.77,1.42
S2,80.651304,11.938790,4.545806,2.805520,0.058579,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.08,2.15,1.88
S3,80.581370,11.945776,4.548465,2.807162,0.117227,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.64,1.27,2.17
S4,80.511354,11.952770,4.551128,2.808805,0.175943,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.72,1.30,2.22
S5,80.441255,11.959771,4.553794,2.810451,0.234728,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.49,1.90,2.09


In [5]:
print(df.shape)

(135, 28)


In [6]:
data_size = df.shape[0]
train_size = int(0.8 * data_size)
test_size = data_size - train_size
nbr_featues = df.shape[1]-3

# The training parameters
Epochs, learning rate ...
<br>There are 3 types of scalers that can be used:
* **'standard' to use StandardScaler**
* **'minmax' to use MinMaxScaler**
* **'robust' to use RobustScaler**


set ***scale_x*** to True to enbale scaling on inputs<br>
set ***scale_y*** to True to enbale scaling on targets

The batch size used here is whole dataset because the dataset is small and will not cause overflowing the memory.

**number_of_tests** is the number of times that we repeat the model training and evaluation to ensure that the aquired results are not exclusive to one experment only.

**loss_each_epoch** after how many epochs the loss will be printed


***Note:*** when using the scaling methods the model will require more epochs.

In [7]:
learning_rate = 0.001
epochs = 1050
batch_size = train_size
show_in_trainig_loss = True
show_losses = True
loss_each_epoch = 100
number_of_tests = 1000
scaler_type = 'standard'
scale_x = False
scale_y = False

In [8]:
x = df.iloc[0:, 0:nbr_featues].values.astype(float)
y = df.iloc[0:, nbr_featues:df.shape[1]].values.astype(float)
x_testing = data.iloc[0:, 0:nbr_featues].values.astype(float)
print(x.shape)
print(y.shape)

(135, 25)
(135, 3)


# MLP structure

<font size="3">The model consists of 3 hidden layers</font>
* The 1st hidden layer has 110 neuron with Sigmoid as activation function to break linearity and to represent newly extracted information in range of 0 and 1.
* The 2nd hidden layer has 100 neuron with Sigmoid as activation function.
* The 3rd hidden layer has 40 neuron with ReLU as activation function, RelU is used here instead of Sigmoid to speed up the training and prevent the vanishing gradient problem that comes from using Simoid in a deep model.


The number of neuron  deacreses as the number of layers increases to get the best representation of input data and the output is has only 3 dimensions<br>
Dropout is used to prevent overfeating the model because the model contains farily big number of neurons and layers in comparison to the small dataset (184 observation).

Adam was used as gradient decent optimizer to speed up the training process

<font size="4">***Note:***&nbsp; *this parameters are addjusted based on experimentation and observation with model and are not 100% explainalbe.*</font>

In [9]:
class MLPRegressor(nn.Module):
    def __init__(self,num_inputs):
        super(MLPRegressor,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            #---------
            nn.Linear(num_inputs, 110),
            nn.Sigmoid(),
            nn.Dropout(p=0.25),
            
            #----------- hidden layer 2 
            nn.Linear(110, 100),
            nn.Sigmoid(),
            nn.Dropout(p=0.15),
            #----------- hidden layer 3 
            nn.Linear(100, 40),
            nn.ReLU(),
            nn.Dropout(p=0.15),
            
            #----------- Output layer
            nn.Linear(40, 3),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [10]:
if scaler_type == 'robust':
    scalerx = RobustScaler()
    scalery = RobustScaler()
elif scaler_type == 'minmax':
    scalerx = MinMaxScaler(feature_range=(0,1))
    scalery = MinMaxScaler(feature_range=(0,1))
elif scaler_type == 'standard':
    scalerx = StandardScaler()
    scalery = StandardScaler()
# fit scaler on data
scalerx.fit(x)
scalery.fit(y)
# apply transform
x_rw = x # 
y_rw = y #
if scale_x:
    x = scalerx.transform(x)
if scale_y:
    y = scalery.transform(y)

tensor_x = torch.from_numpy(x).float()
tensor_y = torch.from_numpy(y).float()
full_dataset = TensorDataset(tensor_x,tensor_y)


# Model evaluation function

<font size="4">**Note: scaling is only beneficial during model training and not evaluating**

Since there are an equivalance between the scaled values and original ones,
When a scaling is used then during the evaluation we transformed the data to its original values so it can be compared to the other published works and to get a better observation on how much the model predicions are shifting from the target values. 

</font>

In [11]:
def evaluate_model(model,loss_function,trainset,testset,scale = False,scaler = None):
    
    def use_scaler(inputs):
        inputs = scaler.inverse_transform(inputs)
        return torch.from_numpy(inputs).float()
    def use_dataset(dataset):
        inps, targets = dataset[:]
        outputs = model(inps)
        
        if scale:
            targets = use_scaler(targets)
            outputs = use_scaler(outputs)
        
        loss = loss_function(outputs, targets)
        error = mean_squared_error(outputs, targets, squared=False,multioutput= 'raw_values')
        return loss, error
    with torch.no_grad():
        model.eval() # switch the model to evaluation mode
        train_loss, train_error = use_dataset(trainset)
        test_loss, test_error = use_dataset(testset)
        model.train() # switch the model to training mode
        return train_error, train_loss, test_loss, test_error
#def test_predict(model):
    #test = np.array([[79.5,20,0,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])
    #print(test.shape)
 #   test = torch.from_numpy(x_testing).float()
#    outputs = mlpRegressor(test)
 #   print(outputs)

# Model training loop

trainig_errors, testing_errors, testing_losses are used to accumulate the aquired results of each training attempt, and later the mean, standard diviation, and minimum resluts will be printed.

In [12]:

trainig_errors = np.zeros((number_of_tests,3))
testing_errors = np.zeros((number_of_tests,3))
testing_losses = np.zeros((number_of_tests,1))


for k in range(number_of_tests):
    mlpRegressor = MLPRegressor(nbr_featues)

    loss_function = nn.MSELoss()
    optimizer = optim.Adam(mlpRegressor.parameters(), lr=learning_rate)
    trainset, testset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
    testloader = DataLoader(testset, batch_size=test_size, shuffle=True)
    for epoch in range(epochs):

        batch_loss = 0

        for i, data in enumerate(trainloader):

            inputs, targets = data

            optimizer.zero_grad()

            outputs = mlpRegressor(inputs)
            #l1_norm = sum(p.abs().sum()
            #            for p in mlpRegressor.parameters())
            #loss = loss_function(outputs, targets) + 0.0003 * l1_norm

            loss = loss_function(outputs, targets)
            loss.backward() # calculate the gradients
            optimizer.step() # chenge the weights
            batch_loss += loss.item()

        if epoch%loss_each_epoch == 0:
            if show_losses and show_in_trainig_loss :
                train_error, train_loss, test_loss, test_error=evaluate_model(mlpRegressor,loss_function,trainset,testset,scale_y,scalery)
                print(f'Epoch: {epoch} | Loss: {train_loss} | testing RMSE: {test_error} | testing MSE: {test_loss}')
                

    train_error, train_loss, test_loss, test_error=evaluate_model(mlpRegressor,loss_function,trainset,testset,scale_y,scalery)
    if show_losses:
        print(f'training RMSE: {train_error} | testing RMSE: {test_error} | testing MSE: {test_loss}')
    #test_predict(mlpRegressor)
    testing_errors[k] = test_error
    trainig_errors[k] = train_error
    testing_losses[k] = test_loss
print(f'training RMSE mean : {trainig_errors.mean(axis=0)} standard diviation {trainig_errors.std(axis=0)} min: {trainig_errors.min(axis=0)} max: {trainig_errors.max(axis=0)}')
print(f'testing RMSE mean : {testing_errors.mean(axis=0)} standard diviation {testing_errors.std(axis=0)} min: {testing_errors.min(axis=0)} max: {testing_errors.max(axis=0)}')
print(f'testing MSE mean : {testing_losses.mean(axis=0)} standard diviation {testing_losses.std(axis=0)} min: {testing_losses.min(axis=0)} max: {testing_losses.max(axis=0)}')

Epoch: 0 | Loss: 11.218615531921387 | testing RMSE: [5.9484353 1.6940424 1.3387693] | testing MSE: 13.34865665435791
Epoch: 100 | Loss: 1.1504247188568115 | testing RMSE: [1.8553708  0.64704674 0.5571628 ] | testing MSE: 1.3905003070831299
Epoch: 200 | Loss: 0.7291083335876465 | testing RMSE: [1.7665417  0.61814076 0.5166531 ] | testing MSE: 1.2565659284591675
Epoch: 300 | Loss: 0.5901669859886169 | testing RMSE: [1.7277662  0.5680698  0.50051564] | testing MSE: 1.1861318349838257
Epoch: 400 | Loss: 0.5049249529838562 | testing RMSE: [1.70938    0.5546113  0.48827028] | testing MSE: 1.1559938192367554
Epoch: 500 | Loss: 0.4304613769054413 | testing RMSE: [1.6734252  0.55546534 0.4901034 ] | testing MSE: 1.1163650751113892
Epoch: 600 | Loss: 0.37640318274497986 | testing RMSE: [1.6617506  0.5344445  0.48515913] | testing MSE: 1.0941417217254639
Epoch: 700 | Loss: 0.34060248732566833 | testing RMSE: [1.6467578  0.5317653  0.48613358] | testing MSE: 1.0769705772399902
Epoch: 800 | Loss: 0

# Results discusion

At first notes we see that:
* The RMSE of omega 2 (first output) in comparision to the others omega 4 and 6 RMSE.

* The RMSE standard diviation of the first output (omega 2) is also high in comparision to the others, it is also worth noting that the model overfits the tarining data fast Thus the model is really sensitive to the selected training and testing data.


The model predictions are sensitive to the trainingset thus we can say that dataset has outliers and have small number of ovebservations in comparision to the number of input dimensions(184x27).

In conclusion the obtained results shows that there is a correlation between the inputs (percentage of componenets) and the targets (JUDD OFELT parameters) but the RMSE of omega 2 is high and not reliable in physics. The dataset is small and the model can be further imporved with more data thus we can say that the current results are not final.

